In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import sys
FOLDERNAME = 'Colab Notebooks/CS 234/cs234'
sys.path.append(f'/content/drive/My Drive/{FOLDERNAME}')


%cd /content/drive/MyDrive/"Colab Notebooks/CS 234/cs234"
! ls # verify that you are in the right directory

/content/drive/MyDrive/Colab Notebooks/CS 234/cs234
alfred	  data_examples  lib	      README.md		setup.py
AutoGPTQ  dockerfile	 LICENSE      requirements.txt	tests
build	  include	 MANIFEST.in  scripts		toolformer
configs   index_files	 nbs	      settings.ini	toolformer.egg-info


In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
! pip install -r requirements.txt
! pip install -e .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.2 MB/s eta 0:00:00
Obtaining file:///content/drive/MyDrive/Colab%20Notebooks/CS%20234/cs234
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.6/973.6 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.0 MB/s et

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# hard bc token ids, maybe need to change YAML?
# model_id = "anudaw/distilled-code-llama"
import torch

model_id = "bigscience/bloom-1b1"
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.13G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [ ]:
from toolformer.data_generator import DataGenerator
from toolformer.api import CalculatorAPI
from toolformer.prompt import calculator_prompt
from toolformer.utils import yaml2dict

config = yaml2dict('./configs/default.yaml')
calculator_api = CalculatorAPI(
    "CalculatorAPI", calculator_prompt,
    sampling_threshold=0.2, filtering_threshold=0.2
)
apis = [calculator_api]
generator = DataGenerator(config, model, tokenizer, apis=apis)

text = "I need to buy 10 cookies for 2 dollars a cookie. The total price is 2 * 10 = 31 dollars"
# text = "From this, we have 10 - 5 minutes = 5 minutes."
task_prompt = "Finish the following text completion. Don't generate more words then are necessary."
augmented_text_ids = generator.generate(text, task_prompt)

text = "I need to buy 10 cookies for 2 dollars a cookie. The total price is 2 * 10 = 31 dollars"
# text = "From this, we have 10 - 5 minutes = 5 minutes."
task_prompt = "Finish the following text completion. Don't generate more words then are necessary."
augmented_text_ids = generator.generate(text, task_prompt)

print(tokenizer.decode(augmented_text_ids[0], skip_special_tokens=True))

I need to buy 10 cookies for 2 dollars a cookie. The total price is 2 * 10 = [Calculator(2 * 10)] 31 dollars

Input: I need to buy 10 cookies for 2 dollars a cookie. The total price.




/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Candidate: I need to buy 10 cookies for 2 dollars a cookie. The total price is 2 * 10 = [CalculatorAPI(2 * 10)]→20$.

Baseline1: I need to buy 10 cookies for 2 dollars a cookie. The total price is 2 * 10 = 31 dollars
Baseline2: I need to buy 10 cookies for 2 dollars a cookie. The total price is 2 * 10 = [CalculatorAPI(2 * 10)] 31 dollars</s>
Respond with True if the Candidate is better than the Baselines, and False otherwise
True
Candidate: I need to buy 10 cookies for 2 dollars a cookie. The total price is [CalculatorAPI(2 * 10)]→20$.

Baseline1: I need to buy 10 cookies for 2 dollars a cookie. The total price is 2 * 10 = 31 dollars
Baseline2: I need to buy 10 cookies for 2 dollars a cookie. The total price is [CalculatorAPI(2 * 10)] 31 dollars.


Respond with True if the Candidate is better than the Baselines, and False otherwise
True
Candidate: I need to buy 10 cookies for 2 dollars a cookie. The total price is 2 * [CalculatorAPI(10 * 2)]→20$.

Baseline1: I need to buy 10 cookies fo

In [7]:
base_weather_key = "API_KEY"

In [28]:
import time
from toolformer.utils import ask_gpt
def ask_gpt_with_backoff(prompts):
  for retry in range(9):
      try:
          response = ask_gpt(prompts)
          return response
      except Exception as e:
          if "529" in str(e):
              print(f"Retrying in {2**retry} seconds due to overload...")
              time.sleep(2**retry)
          else:
              raise e

In [9]:
from toolformer.utils import ask_gpt
from toolformer.prompt import weather_prompt
from toolformer.api import WeatherAPI
from toolformer.gpt import get_input_generator_prompt
from toolformer.utils import ask_gpt, yaml2dict
from toolformer.data_generator import DataGenerator

config = yaml2dict('./configs/default.yaml')
weather_api = WeatherAPI(
    "Weather", weather_prompt, api_key = base_weather_key,
    sampling_threshold=0.2, filtering_threshold=0.2
)

apis = [weather_api]
generator = DataGenerator(config, model, tokenizer, apis=apis)

import csv
import os
filename = f"data_examples/{' '.join([api.name for api in apis])}.csv"
os.makedirs(os.path.dirname(filename), exist_ok=True)

total_samples = 1035
positive_samples = 450
last_milestone = total_samples - (total_samples % 10)

while positive_samples < 1000:
    if total_samples > last_milestone + 10:
        print(f"Total Samples: {total_samples}, Positive Samples: {positive_samples}")
        last_milestone += 10
    example_prompts = weather_prompt[weather_prompt.find ("Input:"):]

    prompts = [{
        "role": "system",
        "content": """You will many examples of Input, Output pairs that will be given for incontext learning to another model.
        Generate more examples of inputs. Do not generate any outputs. Do not include the labels Input and Output.
        Respond with only the examples separated by new lines."""
      }, {
        "role": "user",
        "content": example_prompts
    }]

    prompts = [{
        "role": "user",
        "content": f"""You will many examples of Input, Output pairs that will be given for incontext learning to another model.
        Generate more examples of inputs. Do not generate any outputs. Do not include the labels Input and Output.
        Respond with only the examples separated by new lines. \n{example_prompts}"""
      }]

    response = ask_gpt_with_backoff(prompts)
    texts = response.split("\n")

    for text in texts:
      if len(text) < 2:
        continue
      try:
        augmented_text_ids = generator.generate(text)
        total_samples += max (1, len (augmented_text_ids))
      except Exception as e:
        print(e)
        continue
      with open(filename, "a") as f:
          writer = csv.writer(f)
          # Write header if file is new
          if f.tell() == 0:
              writer.writerow(["input","text","candidate"])
          for augmented_text in augmented_text_ids:
              positive_samples += 1
              decoded_text = tokenizer.decode(augmented_text, skip_special_tokens=True)
              input_generator_prompt = get_input_generator_prompt (decoded_text)
              res = ask_gpt_with_backoff(input_generator_prompt)

              writer.writerow([res, text, decoded_text])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Total Samples: 1044, Positive Samples: 453
Total Samples: 1059, Positive Samples: 457
Total Samples: 1067, Positive Samples: 459
Total Samples: 1077, Positive Samples: 462
index -1 is out of bounds for dimension 0 with size 0
Total Samples: 1086, Positive Samples: 464
index -1 is out of bounds for dimension 0 with size 0
Total Samples: 1096, Positive Samples: 466
Total Samples: 1106, Positive Samples: 468
Total Samples: 1116, Positive Samples: 471
Total Samples: 1126, Positive Samples: 475
Total Samples: 1135, Positive Samples: 479
Total Samples: 1145, Positive Samples: 480
Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}
Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}
Total Samples: 1153, Positive Samples: 480
Retrying in 1 seconds due to overload...
Total Samples: 1163, Positive Samples: 483
Total Samples: 1173, Positive Samples: 484
Total Samples: 1190, Positive Samples: 489
Total 

RateLimitError: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': 'Number of request tokens has exceeded your daily rate limit (https://docs.anthropic.com/en/api/rate-limits); see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase.'}}

In [42]:
# from toolformer.utils import ask_gpt
import csv

file_path = "data_examples/Weather.csv"

import re

def parse_response(response):
    # Define the regular expressions to capture the fields
    input_pattern = re.compile(r'input:\s*(.*)\n')
    text_pattern = re.compile(r'text:\s*(.*)\n')
    candidate_pattern = re.compile(r'candidate:\s*(.*)\n')

    # Find all matches using the regular expressions
    input_matches = input_pattern.findall(response)
    text_matches = text_pattern.findall(response)
    candidate_matches = candidate_pattern.findall(response)

    min_len = min(len(input_matches), len(text_matches), len(candidate_matches))

    # Combine the captured fields into a list of dictionaries
    parsed_data = []
    for i in range(min_len):
        parsed_data.append({
            "input": input_matches[i],
            "text": text_matches[i],
            "candidate": candidate_matches[i]
        })

    return parsed_data

with open(file_path, mode='r') as file:
    reader = list(csv.DictReader(file))
    window_size = 10
    window = [row for row in reader[1:window_size + 1]]
    data = window.copy()
    count = 0
    for row in reader[window_size:]:
        count += 1
        if count % 10 == 0:
            print(f"Processed {count} rows")
        window = window[1:]
        window.append(row)

        examples = '\n'.join([f"input: {row['input']}\n text: {row['text']}\n candidate: {row['candidate']}" for row in window])
        messages = [
            {
                "role": "user",
                "content": f"""
                Generate weather-related responses based on the given inputs. The first response labeled text should be generic, and the second response labeled candidate should include a realistic weather condition provided by the API and any relevant advice or forecast details.
                Generate responses labeled input, text, and candidate and nothing else.
                {examples}"""
            }
        ]
        response = ask_gpt_with_backoff(messages)

        data = parse_response(response)
        if len(data) == 0:
            continue

        new_file_path = "data_examples/Weather_augmented.csv"

        with open(new_file_path, "a") as f:
            writer = csv.writer(f)
            # Write header if file is new
            if f.tell() == 0:
                writer.writerow(["input","text","candidate"])
            for d in data:
                writer.writerow([d['input'], d['text'], d['candidate']])

Processed 10 rows
Processed 20 rows


KeyboardInterrupt: 